In [32]:
import nltk, re, pprint
from nltk import word_tokenize

## Part 1: Processing real text (from out on the inter-webs)

## Part 2: Community detection

In [33]:
#  Loading the graph from Github as decribed in the exercise
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import requests
import random
import re


# load the List_of_mainstream_rock_performers.txt file
with open('List_of_mainstream_rock_performers.txt', 'r') as file:
    data = file.read()

# Regex to match wiki-links in data
pattern = r'\[\[(.*?)\]\]'
# Find mathes
matches = re.findall(pattern, data)

bands = matches
print(f'There are {len(bands)} bands in the list.')

# remove all bands named "AllMusic"
bands = [band for band in bands if "AllMusic" not in band]
print(f'After removal, there are {len(bands)} bands in the list.')



There are 489 bands in the list.
After removal, there are 488 bands in the list.


In [34]:
# Extracted the bandnames used in the wiki links. 

# replace the spaces in band names with underscores and store in new list
bands_underscored = [band.replace(" ", "_") for band in bands]
# Remove all "|" and anything after it in band names and store in new list
bands_cleaned = [band.split('|')[0] for band in bands_underscored]
print(bands_cleaned)

# function to sanitize filenames, so that they can be used as valid filenames
def sanitize_filename(name):
    # Replace any character that is not a letter, number, underscore, or hyphen with an underscore
    return re.sub(r'[^\w\-]', '_', name)

# Sanitize band names for filenames
bands_sanitized = [sanitize_filename(band) for band in bands_cleaned] # used in filenames

['10cc', '10_Years_(band)', '3_Doors_Down', '311_(band)', '38_Special_(band)', 'ABBA', 'Accept_(band)', 'AC/DC', 'Bryan_Adams', 'Aerosmith', 'AFI_(band)', 'Air_Supply', 'The_Alan_Parsons_Project', 'Alice_in_Chains', 'The_All-American_Rejects', 'The_Allman_Brothers_Band', 'Alter_Bridge', 'Ambrosia_(band)', 'America_(band)', 'The_Animals', 'Adam_Ant', 'Anthrax_(American_band)', 'April_Wine', 'Arcade_Fire', 'Arctic_Monkeys', 'Asia_(band)', 'Audioslave', 'Avenged_Sevenfold', 'Awolnation', "The_B-52's", 'Bachman–Turner_Overdrive', 'Bad_Company', 'Badfinger', 'The_Band', 'The_Bangles', 'Barenaked_Ladies', 'Bay_City_Rollers', 'The_Beach_Boys', 'Beastie_Boys', 'The_Beatles', 'Bee_Gees', 'Beck', 'Ben_Folds_Five', 'Pat_Benatar', 'Chuck_Berry', 'The_Big_Bopper', 'Billy_Talent', 'The_Black_Crowes', 'The_Black_Keys', 'Black_Sabbath', 'Black_Stone_Cherry', 'Black_Veil_Brides', 'Blink-182', 'Blondie_(band)', 'Bloodhound_Gang', 'Blue_October', 'Blue_Öyster_Cult', 'Blues_Traveler', 'James_Blunt', 'Blur

In [37]:
import os
import re
import json
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

def extract_genres_from_wikitext(wikitext, artist_name, debug=False):
    """
    Parses wikitext to find and clean genres, now with stricter filtering
    to remove non-genre junk like URLs and template fragments.
    """
    if not isinstance(wikitext, str):
        return None

    # Step 1: Find the entire infobox.
    infobox_match = re.search(r'\{\{(Infobox musical artist|Infobox band|Infobox person)(.*?)\}\}', wikitext, re.DOTALL | re.IGNORECASE)
    if not infobox_match:
        return None
    infobox_content = infobox_match.group(2)

    # Step 2: Find the genre field using our robust method.
    genre_field_match = re.search(r'\n\s*\|\s*genre\s*=', infobox_content, re.IGNORECASE)
    if not genre_field_match:
        return None

    text_after_genre = infobox_content[genre_field_match.end():]
    next_field_match = re.search(r'\n\s*\|', text_after_genre)
    
    if next_field_match:
        raw_genres = text_after_genre[:next_field_match.start()]
    else:
        raw_genres = text_after_genre

    # Step 3: Initial cleanup (this part is the same).
    # IMPROVED: More robustly remove reference tags, including self-closing ones.
    #raw_genres = re.sub(r'<ref>.*?</ref>', '', raw_genres, flags=re.DOTALL)
    raw_genres = re.sub(r'\{\{cite.*?\}\}', '', raw_genres, flags=re.DOTALL)
    raw_genres = re.sub(r'<!--.*?-->', '', raw_genres, flags=re.DOTALL)
    raw_genres = re.sub(r'<ref.*?>.*?</ref>', '', raw_genres, flags=re.DOTALL)
    raw_genres = re.sub(r'<.*?>', '', raw_genres)
    raw_genres = re.sub(r'\[\[(?:[^|\]]*\|)?([^\]]+)\]\]', r'\1', raw_genres)
    # More aggressive template removal
    raw_genres = raw_genres.replace('}}', '')

    # Step 4: Split into a list of potential genres.
    genres = re.split(r'[,/\n*]', raw_genres)
    print(f"Raw genres for {artist_name}: {genres}")

    # --- NEW & IMPROVED: FINAL FILTERING ---
    cleaned_genres = []
    # List of junk words to ignore. Can be expanded if you see more.
    junk_words = {'artist', 'web', 'website', 'title', 'url', 'cite'} 

    for genre in genres:
        genre = genre.strip().lower()
        
        # --- Quality Control Checks ---
        if not genre:
            continue  # Skip empty strings that result from splitting.
        if genre in junk_words:
            continue  # Skip common junk words.
        if '.' in genre and ('com' in genre or 'org' in genre or 'gov' in genre):
            continue  # Skip things that look like URLs.
        if any(char in genre for char in ['|', '=', '{', '}']):
            continue  # Skip leftover template fragments.
        if len(genre) > 25:
            continue # Skip overly long strings which are likely errors.
        
        # --- End of Quality Control ---

        # Standardize "rock and roll"
        if 'rock' in genre and ('&' in genre or 'and' in genre) and 'roll' in genre:
            genre = 'rock and roll'
        
        cleaned_genres.append(genre)

    return cleaned_genres if cleaned_genres else None


# --- Main Script Execution (No changes needed here) ---

bands_folder_path = r"/Users/iben/Desktop/Social graphs/SocialGraphs/Iben/Week7/bands"
artist_genres_map = {}

if not os.path.isdir(bands_folder_path):
    print(f"Error: The directory '{bands_folder_path}' does not exist.")
else:
    print("Running with enhanced filtering logic...")
    for filename in os.listdir(bands_folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(bands_folder_path, filename)
            artist_name = os.path.splitext(filename)[0].replace('_', ' ')

            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                wikitext = None
                pages = data.get('query', {}).get('pages', {})
                if pages:
                    page_id = next(iter(pages))
                    page = pages[page_id]
                    if 'revisions' in page and page['revisions'] and 'slots' in page['revisions'][0]:
                        slots = page['revisions'][0]['slots']
                        if 'main' in slots and '*' in slots['main']:
                             wikitext = slots['main']['*']
                
                if wikitext:
                    genres = extract_genres_from_wikitext(wikitext, artist_name)
                    if genres:
                        artist_genres_map[artist_name] = genres

            except Exception as e:
                print(f"An unexpected error occurred with file {filename}: {e}")

Running with enhanced filtering logic...
Raw genres for Funkadelic: [' {{hlist|Funk rock{{cite book|title= Encyclopedia of Music in the 20th Century |editor-first= Lol |editor-last= Henderson |editor-first2= Lee |editor-last2= Stacey |publisher= Routledge |year= 2013 |isbn= 978-1-57958-079-7 |chapter= Funk|page=231']
Raw genres for Slayer: [' Thrash metal']
Raw genres for Great White: [' {{flatlist|', '', ' Hard rock', '', ' glam metal{{Cite book|last=Phillips|first=William|title=Encyclopedia of heavy metal music|date=2009|others=Brian Cogan|isbn=978-0-313-34801-3|location=Westport', ' Connecticut|oclc=475534546|pages=102']
Raw genres for The Dave Clark Five: [' {{Hlist|Rock and roll|beat|pop']
Raw genres for Anthrax  American band : [' {{flatlist|', '', ' Thrash metal', '', ' heavy metal', '', ' groove metal', '', ' {{nowrap|alternative metal']
Raw genres for Flogging Molly: [' Celtic punk', ' folk punk']
Raw genres for Simple Plan: [' ', '{{flatlist|', '', ' Pop-punk', '', ' alternat

In [38]:
# print all genres
for artist, genres in artist_genres_map.items():
    print(f"{artist}: {', '.join(genres)}")


Slayer: thrash metal
Great White: hard rock
Anthrax  American band : thrash metal, heavy metal, groove metal
Flogging Molly: celtic punk, folk punk
Simple Plan: pop-punk, alternative rock, pop rock, power pop, emo
AC DC: hard rock, blues rock, rock and roll, heavy metal
Hoobastank: alternative rock
Limp Bizkit: nu metal, rap rock, rap metal
Midnight Oil: 2013, 05, 20130713000619, http:, 2013, 05
Awolnation: alternative rock, electronic rock
Thirty Seconds to Mars: alternative rock
The Pretenders: pretenders-mn0000492331
10cc: articles
White Zombie  band : groove metal, industrial metal, alternative metal, noise rock (early)
Billy Talent: punk rock, alternative rock
Starset: alternative rock, electronic rock
Marilyn Manson  band : industrial rock, industrial metal, gothic rock, alternative metal
Korn: nu metal, alternative metal
Dire Straits: roots rock, blues rock, pub rock
The Moody Blues: rockers, 20180101142903, https:
Garbage  band : alternative rock, electronic rock
Our Lady Peace